First Training with the GPT-2 decoder 🤖
-----------------------------------

In this notebook, we will train the pre-trained GPT-2 model provided by OPEN-AI. It only tests how the model can be accurate on the corpora we extracted. It is undoubtedly a partial model. For the prediction, we will follow the fine-tuning tutorial available at the following medium link [fine-tuning-transformers](https://medium.com/towards-data-science/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e) and add hyperparameter search with the `wandb` library.


We will make this with and without augmentation and see where we obtain better results. 

In [55]:
# define environment
%env WANDB_PROJECT=gpt2_french_wolof_sweeps
%env WANDB_LOG_MODEL=true
%env WANDB_NOTEBOOK_NAME=training_gpt2_2.ipynb
%env WANDB_API_KEY=237a8450cd2568ea1c8e1f8e0400708e79b6b4ee

env: WANDB_PROJECT=gpt2_french_wolof_sweeps
env: WANDB_LOG_MODEL=true
env: WANDB_NOTEBOOK_NAME=training_gpt2_2.ipynb
env: WANDB_API_KEY=237a8450cd2568ea1c8e1f8e0400708e79b6b4ee


In [42]:
!pip install -qq wandb --upgrade

In [43]:
# !pip install evaluate -qq
# !pip install sacrebleu -qq
# !pip install optuna -qq
# !pip install transformers -qq 
# !pip install tokenizers -qq
# !pip install nlpaug -qq
# !pip install ray[tune] -qq

In [56]:
# let us import all necessary libraries
from wolof_translate.utils.sent_transformers import TransformerSequences
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
from wolof_translate.data.dataset_v1 import SentenceDataset
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np
import evaluate
import torch


We will create two models: 

- One translating the french corpus to a wolof corpus [french_to_wolof](#french-to-wolof)
- One translating the wolof corpus to a french corpus [wolof_to_french](#wolof-to-french)

--------------

## French to wolof

### Configure dataset 🔠

We can use the same custom dataset that we created in [text_augmentation](text_augmentation.ipynb). But we need to split the data between train and test sets and save them.

In [57]:
# # load the corpora and split into train and test sets
# corpora = pd.read_csv("sent_extraction.csv")

# train_set, test_set = train_test_split(corpora, test_size=0.1, random_state=50)

# # let us save the sets
# train_set.to_csv("train_set.csv", index=False)

# test_set.to_csv("data/extractions/new_data/test_set.csv", index=False)

Let us recuperate the datasets with and without augmentation.

In [58]:
# without augmentation
train_dataset = SentenceDataset("data/extractions/new_data/train_set.csv")

test_dataset = SentenceDataset("data/extractions/new_data/test_set.csv")

# with augmentation
fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2),
                                       remove_mark_space, delete_guillemet_space)

wf_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2),
                                       remove_mark_space, delete_guillemet_space)

train_dataset_aug = SentenceDataset("data/extractions/new_data/train_set.csv", 
                                cp1_transformer=fr_augmentation, truncation=True,
                                cp2_transformer=wf_augmentation, max_len=579)

test_dataset_aug = SentenceDataset("data/extractions/new_data/test_set.csv",
                               cp1_transformer=fr_augmentation, truncation=True,
                               cp2_transformer=wf_augmentation, max_len=579)

### Configure hyperparameter search ⚙️

We have to configure the search space and the search method ("random" in our case). .

In [59]:
import wandb
wandb.login(key="237a8450cd2568ea1c8e1f8e0400708e79b6b4ee")

# hyperparameters
sweep_config = {
    'method': 'random',
    'metric':
    {
      'goal': 'minimize',
      'name': 'eval_loss'
    },
    'parameters':
    {
      'epochs': {
          'value': 1
      },
      'batch_size': {
          'values': [2, 3, 5]
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 1e-5,
          'max': 1e-3
      },
      'weight_decay': {
          'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
      },
      'dataset_aug': {
          'values': ['True', 'False']
      }
    }
}

# Initialize the hyperparameter search
sweep_id = wandb.sweep(sweep_config, project = "gpt2-french-wolof-sweeps")



wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Oumar Kane/.netrc


Create sweep with ID: guoc9d57
Sweep URL: https://wandb.ai/oumar-kane-team/gpt2-french-wolof-sweeps/sweeps/guoc9d57


### Configure the model and the evaluation function ⚙️

Let us recuperate the model and resize the token embeddings.

In [60]:
def gpt2_model_init():
  # set the mode name
  model_name = "gpt2"

  # recuperate the tokenizer from the dataset
  tokenizer = train_dataset.tokenizer

  # configure the model
  model = GPT2LMHeadModel.from_pretrained(model_name).cuda()

  # resize the token embeddings
  model.resize_token_embeddings(len(tokenizer))

  return model

Let us evaluate the predictions with the `bleu` metric.

In [61]:
# %%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):
        
        preds, labels = eval_preds.preds.detach().cpu(), labels.detach().cpu()
        
        if isinstance(preds, tuple):
            
            preds = preds[0]
        
        if self.decoder is None:
            
            decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)
            
            decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)
            
            result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
            
            result = {"bleu": result["score"]}
            
            prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in preds]
            
            result["gen_len"] = np.mean(prediction_lens)
        
        else:
            
            predictions = list(self.decoder(preds))
            
            labels = list(self.decoder(labels))
      
            decoded_preds, decoded_labels = self.postprocess_text(predictions, labels)
            
            result = self.metric.compute(predictions=predictions, references=labels)
            
            result = {"bleu": result["score"]}
        
        result = {k:round(v, 4) for k, v in result.items()}

        wandb.log("bleu", result["bleu"])
            
        return result

In [62]:
# %run wolof-translate/wolof_translate/utils/evaluation.py

Let us initialize the evaluation object.

In [63]:
translation_eval = TranslationEvaluation(test_dataset.tokenizer)

### Searching for the best parameters 🕖

Let us define the data collator.

In [64]:
def data_collator(batch):
    """Generate a batch of data to provide to trainer

    Args:
        batch (_type_): The batch

    Returns:
        dict: A dictionary containing the ids, the attention mask and the labels
    """
    input_ids = torch.stack([b[0] for b in batch])
    
    attention_mask = torch.stack([b[1] for b in batch])
    
    labels = torch.stack([b[0] for b in batch])
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask,
            'labels': labels}

Let us initialize the training arguments and make random search.

In [65]:
%%wandb

def train(config = None):

  with wandb.init(config = config):
    
    global train_dataset

    global test_dataset

    # seed
    torch.manual_seed(50)

    # set sweep configuration
    config = wandb.config

    if config.dataset_aug == 'True':

      train_dataset = train_dataset_aug

      test_dataset = test_dataset_aug

    # set training arguments
    training_args = TrainingArguments(f"{path}training1/results",
                                      report_to = "wandb",
                                      num_train_epochs=config.epochs,
                                      # logging_steps=100,
                                      load_best_model_at_end=True,
                                      save_strategy="epoch",
                                      evaluation_strategy="epoch",
                                      logging_strategy = 'epoch',
                                      per_device_train_batch_size=config.batch_size, 
                                      per_device_eval_batch_size=3,
                                      learning_rate=config.learning_rate,
                                      weight_decay=config.weight_decay,
                                      logging_dir=f'{path}gpt2_training_logs1',
                                      remove_unused_columns = False,
                                      fp16 = True,
                                      )   

    # define training loop
    trainer = Trainer(model_init=gpt2_model_init,
                      args=training_args,
                      train_dataset=train_dataset, 
                      eval_dataset=test_dataset,
                      data_collator=data_collator,
                      compute_metrics=translation_eval.compute_metrics
                      )

    # start training loop
    trainer.train()

wandb.agent(sweep_id, train, count = 10)


wandb: Agent Starting Run: 5aoyf0bi with config:
wandb: 	batch_size: 3
wandb: 	dataset_aug: True
wandb: 	epochs: 1
wandb: 	learning_rate: 1.3094777168952637e-05
wandb: 	weight_decay: 0.1


Run 5aoyf0bi errored: NameError("name 'path' is not defined")
wandb: ERROR Run 5aoyf0bi errored: NameError("name 'path' is not defined")
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: kyyt30ov with config:
wandb: 	batch_size: 2
wandb: 	dataset_aug: False
wandb: 	epochs: 1
wandb: 	learning_rate: 3.3559839165982405e-05
wandb: 	weight_decay: 0.5


Run kyyt30ov errored: NameError("name 'path' is not defined")
wandb: ERROR Run kyyt30ov errored: NameError("name 'path' is not defined")
wandb: Agent Starting Run: fhznobej with config:
wandb: 	batch_size: 3
wandb: 	dataset_aug: False
wandb: 	epochs: 1
wandb: 	learning_rate: 3.720476295351395e-05
wandb: 	weight_decay: 0.2


Run fhznobej errored: NameError("name 'path' is not defined")
wandb: ERROR Run fhznobej errored: NameError("name 'path' is not defined")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [13]:
# let us get the best model
trainer.state.best_model_checkpoint

'data/training1/results/run-9/checkpoint-368'

In [ ]:
# load from a checkpoint and continue the training
# trainer._load_from_checkpoint('data/training1/results/checkpoint-734/')



We see that the model is over-fitted. We must fine-tune the model and augment it to add some noise into the training step.

### Predictions

Let us generate texts and store into a DataFrame.

In [ ]:

# set the model to eval mode
_ = model.eval()

# run model inference on all test data
original_traduction, predicted_traduction, original_text, scores = [], [], [], {}

for data in tqdm(DataLoader(test_dataset)):
    
    # recuperate the two part of the sentence
    sents = list(test_dataset.decode(data[0]))
    
    cp1_sent, cp2_sent = sents[0][0], sents[0][1] 
    
    # create the sentence to traduce
    sent1 = f'{test_dataset.cls_token}{cp1_sent}{test_dataset.sep_token}'
    
    # generate tokens
    encoding = tokenizer(sent1, return_tensors='pt')
    
    generated = encoding.input_ids.cuda()
    
    attention_mask = encoding.attention_mask.cuda()
    
    # recuperate the pad token id
    pad_token_id = tokenizer.pad_token_id
    
    # perform prediction
    sample_outputs = model.generate(generated, do_sample = False, top_k = 50, max_length = test_dataset.max_len, top_p = 0.90,
                                    temperature = 0, num_return_sequences = 0, attention_mask = attention_mask, pad_token_id = pad_token_id)
    
    # calculate the score and add it to the score
    result = translation_eval.compute_metrics((sample_outputs, generated))
    
    if not scores: scores.update({k: v for k, v in result.items()})
    
    else: scores.update({k: round((scores[k] + v) / 2, 4) for k, v in result.items()})
    
    # decode the predicted tokens into texts
    sent2 = list(test_dataset.decode(sample_outputs, True))[0]
    
    print(sent2)
    # append results
    original_traduction.append(cp2_sent)
    predicted_traduction.append(sent2)
    original_text.append(cp1_sent)

# transform result into data frame
df_ft_to_wf = pd.DataFrame({'original_text': original_text,
                            'original_label': original_traduction,
                            'predicted_label': predicted_traduction})

# print the result
df_ft_to_wf.head()

  1%|          | 1/82 [00:06<09:20,  6.91s/it]

Muy nag, nag, nag, nag, nag, nag, nag, nag, nag, nag, ci sama yaay daan wax dëgg-dëgg, ci sama biir yu kowe ya.


  2%|▏         | 2/82 [00:12<08:10,  6.13s/it]

Nettali sax, su ma ci sama yaay moom, di ma ni sama yaay daan nettali nii, di ma koy jekk-xare, di sama yaay daan nettali nii. Mu ni sama yaay daan nettali nii, sama yaay daan nettali nii, sama yaay daan nettali nii.


  4%|▎         | 3/82 [00:18<08:17,  6.30s/it]

...

 99%|█████████▉| 81/82 [17:56<00:25, 25.53s/it]

Baay na ko ko ko daan def, di ko ci sama yaay daan def.


100%|██████████| 82/82 [18:23<00:00, 13.45s/it]

Nettali sax, du dox ci seen biir yu dóomu-gisu Baay, seen biir yu dóomu-gisu Baay, seen biir yu mag, seen biir yu mag, seen biir yu mag, seen biir yu ñu ànd ak seen biir yu ñu ànd ak seeni dàll, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp, seen bopp seen bopp seen bopp, seen bopp, seen bopp, seen bopp seen bopp, seen bopp seen bopp, seen bopp seen bopp seen bopp, seen bopp, seen bopp seen bopp, seen bopp seen bopp, seen bopp, seen bopp seen bopp, seen bopp, seen bopp, seen bopp seen bopp, seen bopp, seen bopp seen bopp, seen bopp seen bopp, see

,original_text,original_label,predicted_label
0,"Hommes, femmes, enfants sont pris dans un pièg...","Mag, ndaw, góor ak jigéen jàq, song àll bi te ...","Muy nag, nag, nag, nag, nag, nag, nag, nag, na..."
1,"Les portes ouvertes à l'émigration, ces cohort...","Loolu doyul, biñ duggee ci atum 60, ñu ubbil A...","Nettali sax, su ma ci sama yaay moom, di ma ni..."
2,"Les sulfamides sont rares, les poudres et les ...","Garab yu néew lañuy yóbbale, diy puudar ak i t...","Ngalla, su ma sama yaay moom, sama yaay moom, ..."
3,Je peux ressentir l'émotion qu'il éprouve à tr...,"Li koy yëngal noonu, xam naa ko. Lan moo ko dà...","Nettali askan wi, ay way-jur, ay ndell, ay nde..."
4,"Tout ce que j'ai pu savoir de cette période, c...",Sunu yaay daal moo daan sànni léeg-léeg ci wax...,"Nettali sax, du sax, du sax, du sax, du sax, d..."


Like we specified in conclusion of the first processing on the corpora, some words appear a too important number of times in the text and we can see that their arrive in the large part of the translations. We can add augmentation and see if we obtain better results on the evaluation set.

### Training with augmentation

We must initialize a new model.

In [ ]:
# set the mode name
model_name = "gpt2"

# recuperate the tokenizer from the dataset
tokenizer = train_dataset_aug.tokenizer

# configure the model
model_aug = GPT2LMHeadModel.from_pretrained(model_name).cuda()

# resize the token embeddings
model_aug.resize_token_embeddings(len(tokenizer))


Embedding(15696, 768)

Let us initialize the training arguments and fine-tune the model.

In [ ]:
# seed
torch.manual_seed(50)

# create training arguments
training_args = TrainingArguments("data/training1/results_aug",
                                  overwrite_output_dir=True,
                                  num_train_epochs=3,
                                  logging_steps=100,
                                  load_best_model_at_end=True,
                                  save_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=2, 
                                  per_device_eval_batch_size=2,
                                  learning_rate=2e-5,
                                #   warmup_steps=120,
                                  weight_decay=0.05,
                                  logging_dir='gpt2_training_aug_logs1')

# start training
trainer = Trainer(model=model_aug,
                  args=training_args,
                  train_dataset=train_dataset_aug, 
                  eval_dataset=test_dataset_aug,
                  data_collator=data_collator,
                  # compute_metrics=translation_eval.compute_metrics
                  )

# def optuna_hp_space(trial):
  
#   return {
#     "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log = True),
#     "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
#     "seed": trial.suggest_int("seed", 1, 40),
#     "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 3, 4, 5, 6])
#   }

# # Searching for the best hyperparameters
# trainer.hyperparameter_search(
#   direction = "maximize",
#   backend = "optuna",
#   hp_space = optuna_hp_space,
#   n_trials = 10,
# )


In [ ]:
# load from a checkpoint and continue the training
# trainer._load_from_checkpoint('data/training1/results/checkpoint-734/')

trainer.train()

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|▏         | 15/1101 [00:29<31:12,  1.72s/it] 

-----------------------

### Colab download step

In [19]:
!zip -r /content/data.zip /content/data

NotImplementedError: ignored

------------------